In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterstats
import util

In [2]:
#import calliope zones
calliope_zones=gpd.read_file(snakemake.input["calliope_zones"])

#import gas demand per zone
gas_per_zone=pd.read_csv(snakemake.input["gas_zones"], index_col=0)

#import nuts3 zones
nuts3=gpd.read_file(snakemake.input["nuts3_shapefile"])
nuts3=nuts3.loc[nuts3['CNTR_CODE'] =='UK']
nuts3.set_index('NUTS_ID', inplace=True)

#import population raster
with rasterio.open(snakemake.input["pop_uk"]) as pop_grid:
    array = pop_grid.read(1)
    crs = pop_grid.crs
    affine = pop_grid.transform
#import dwelling data from eurostats
nuts3_dwelling= util.load_eurostat_tsv(snakemake.input["nuts3_dwelling"],
                                  index_names=["housing","building","unit", "time"])

In [3]:
#select data from UK and convert to single index
nuts3_dwelling.reset_index(inplace=True)
nuts3_dwelling.query('housing == "DW"', inplace=True)
nuts3_dwelling=nuts3_dwelling.loc[:,nuts3_dwelling.columns.str.contains("UK")].T
nuts3_dwelling.index= nuts3_dwelling.index.str.strip()

In [4]:
#sum up dwellings by single or multi-family
nuts3_dwelling['RES']= nuts3_dwelling.iloc[:, 1]
nuts3_dwelling['SFH']= nuts3_dwelling.iloc[:, 2]
nuts3_dwelling['MFH']= nuts3_dwelling.iloc[:, 3]+ nuts3_dwelling.iloc[:, 4]
nuts3_dwelling=nuts3_dwelling.dropna()

In [6]:
#join dwelling stats with shapefile
nuts3_dwelling = nuts3_dwelling.join(nuts3)
nuts3_dwelling = nuts3_dwelling[nuts3_dwelling['geometry'].notnull()]
nuts3_dwelling = gpd.GeoDataFrame(
    nuts3_dwelling, geometry='geometry', crs=crs)

In [7]:
#overlay 
nuts3_dwelling = gpd.overlay(calliope_zones.to_crs(crs), nuts3_dwelling.to_crs(crs), how='intersection', keep_geom_type=False)

#calculate population of each snippet
pop=rasterstats.zonal_stats(nuts3_dwelling.to_crs(crs), array, affine=affine, stats='sum', nodata=0)
nuts3_dwelling['population']= [i['sum'] for i in pop]
 
#calculate weighted average
nuts3_dwelling = nuts3_dwelling.set_index(['Name_1', 'NUTS_NAME'], drop=True)
weights = nuts3_dwelling.population.div(nuts3_dwelling.sum(level='NUTS_NAME').population)
nuts3_dwelling['SFH_weighted'] = weights*nuts3_dwelling['SFH']
nuts3_dwelling['MFH_weighted'] = weights*nuts3_dwelling['MFH']

zones_dwelling=nuts3_dwelling.groupby(level='Name_1').sum()


In [8]:
#calculate ratios
zones_dwelling['RES_TOT']= zones_dwelling['SFH_weighted']+zones_dwelling['MFH_weighted']
zones_dwelling['Ratio_SFH']=zones_dwelling['SFH_weighted']/zones_dwelling['RES_TOT']
zones_dwelling['Ratio_MFH']=zones_dwelling['MFH_weighted']/zones_dwelling['RES_TOT']

#merge dwelling and gas consumption data
zones_dwelling['domestic_weighted'] = gas_per_zone['domestic_weighted']
zones_dwelling['non-domestic_weighted']= gas_per_zone['non-domestic_weighted']

#calculate gas consumption by house type
zones_dwelling['gas_SFH']=zones_dwelling['domestic_weighted']*zones_dwelling['Ratio_SFH']
zones_dwelling['gas_MFH']=zones_dwelling['domestic_weighted']*zones_dwelling['Ratio_MFH']

#export to csv
zones_dwelling_gas=zones_dwelling.loc[:,['non-domestic_weighted', 'gas_SFH', 'gas_MFH']]
zones_dwelling_gas.to_csv(snakemake.output[0])